## Discrete Event Simulation
## Bank System 

In [1]:
import random
import simpy

In [2]:
RANDOM_SEED = 42
NEW_CUSTOMERS = 5  # Total number of customers
INTERVAL_CUSTOMERS = 10.0  # Generate new customers roughly every x seconds
MIN_PATIENCE = 1  # Min. customer patience
MAX_PATIENCE = 3  # Max. customer patience

In [3]:
def customer(env, name, counter, time_in_bank):
    """Customer arrives, is served and leaves."""
    arrive = env.now
    print('%7.4f %s: Here I am' % (arrive, name))

    with counter.request() as req:
        patience = random.uniform(MIN_PATIENCE, MAX_PATIENCE)
        # Wait for the counter or abort at the end of our tether
        results = yield req | env.timeout(patience)

        wait = env.now - arrive

        if req in results:
            # We got to the counter
            print('%7.4f %s: Waited %6.3f' % (env.now, name, wait))

            tib = random.expovariate(1.0 / time_in_bank)
            yield env.timeout(tib)
            print('%7.4f %s: Finished' % (env.now, name))

        else:
            # We reneged
            print('%7.4f %s: RENEGED after %6.3f' % (env.now, name, wait))

In [4]:
def source(env, number, interval, counter):
    """Source generates customers randomly"""
    for i in range(number):
        c = customer(env, 'Customer%02d' % i, counter, time_in_bank=12.0)
        env.process(c)
        t = random.expovariate(1.0 / interval)
        yield env.timeout(t)

In [9]:
# Setup and start the simulation
print('Bank System')
random.seed(RANDOM_SEED)
env = simpy.Environment()

Bank System


In [10]:
# Start processes and run
counter = simpy.Resource(env, capacity=1)
env.process(source(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter))
env.run()

 0.0000 Customer00: Here I am
 0.0000 Customer00: Waited  0.000
 3.8595 Customer00: Finished
10.2006 Customer01: Here I am
10.2006 Customer01: Waited  0.000
12.7265 Customer02: Here I am
13.9003 Customer02: RENEGED after  1.174
23.7507 Customer01: Finished
34.9993 Customer03: Here I am
34.9993 Customer03: Waited  0.000
37.9599 Customer03: Finished
40.4798 Customer04: Here I am
40.4798 Customer04: Waited  0.000
43.1401 Customer04: Finished


In [42]:
import random
import simpy

In [3]:
INTERVAL_CUSTOMERS = 7.0
waiting_time = 0

In [82]:
def Customer(env, name, counter, time_in_Takeout ):
    """Customer arrives, is served and leaves."""
    arrive = env.now
    print('%s arrives at Takeout at %4.2f ' % (name,arrive))

    with counter.request() as req:

        wait = env.now - arrive
        global waiting_time 
        waiting_time += wait
        results = yield req

            
        print('%s enters Takeout at  %4.2f' % (name, env.now+wait))


        tib = 5
        yield env.timeout(tib)
        ran = random.uniform(50,99)
        print("Takeout fulfilled","%5.2f"%ran ,"% of", name,"s appetite.")
        print('%s: leaves Takeout at %4.2f ' % ( name,env.now))


In [83]:
def Source(env, number, interval, counter):
    """Source generates customers randomly"""
    c1 = Customer(env, 'Customer 0', counter, 5)
    c2 = Customer(env, 'Customer 1', counter, 5)
    c3 = Customer(env, 'Customer 2', counter, 5)
    c4 = Customer(env, 'Customer 3', counter, 5)
    env.process(c1)
    env.process(c2)
    env.process(c3)
    env.process(c4)
    i = 4
    while(i):
        c = Customer(env, 'Customer%d' % i, counter, 5)
        env.process(c)
        t = random.expovariate(1.0 / interval)
        i+=1
        yield env.timeout(7)
        

In [84]:
env = simpy.Environment()

In [85]:
counter = simpy.Resource(env, capacity=2)
env.process(Source(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter))
env.run(until=20)
print("Total waiting time :",waiting_time)

Customer 0 arrives at Takeout at 0.00 
Customer 1 arrives at Takeout at 0.00 
Customer 2 arrives at Takeout at 0.00 
Customer 3 arrives at Takeout at 0.00 
Customer4 arrives at Takeout at 0.00 
Customer 0 enters Takeout at  0.00
Customer 1 enters Takeout at  0.00
Takeout fulfilled 61.28 % of Customer 0 s appetite.
Customer 0: leaves Takeout at 5.00 
Takeout fulfilled 60.79 % of Customer 1 s appetite.
Customer 1: leaves Takeout at 5.00 
Customer 2 enters Takeout at  5.00
Customer 3 enters Takeout at  5.00
Customer5 arrives at Takeout at 7.00 
Takeout fulfilled 80.92 % of Customer 2 s appetite.
Customer 2: leaves Takeout at 10.00 
Takeout fulfilled 61.22 % of Customer 3 s appetite.
Customer 3: leaves Takeout at 10.00 
Customer4 enters Takeout at  10.00
Customer5 enters Takeout at  10.00
Customer6 arrives at Takeout at 14.00 
Takeout fulfilled 92.12 % of Customer4 s appetite.
Customer4: leaves Takeout at 15.00 
Takeout fulfilled 53.47 % of Customer5 s appetite.
Customer5: leaves Takeout a

In [4]:
INTERVAL_CUSTOMERS = 7.0

In [103]:
class Takeout(object):
        
    def __init__(self, env, name, counter, time_in_Takeout):
        self.env = env
        self.time_in_Takeout = time_in_Takeout
        self.counter = counter
        self.name = name
        self.waiting_time = 0
        self.action = env.process(self.run())
                

    def run(self):
        """Customer arrives, is served and leaves."""
        arrive = env.now
        print('%s arrives at Takeout at %4.2f ' % (self.name,arrive))

        with counter.request() as req:

            wait = env.now - arrive
            self.waiting_time += wait
            print('%s enters Takeout at  %4.2f' % (self.name, env.now+wait))


            tib = 5
            yield self.env.timeout(tib)
            ran = random.uniform(50,99)
            print("Takeout fulfilled","%5.2f"%ran ,"% of", self.name,"s appetite.")
            print('%s: leaves Takeout at %4.2f ' % ( self.name,env.now))



In [140]:
env = simpy.Environment()

In [141]:
def Source(env, number, interval, counter):
    """Source generates customers randomly"""
    c = Takeout(env, 'Customer%d' % 0, counter, 5)
    c1 = Takeout(env, 'Customer%d' % 1, counter, 5)
    c2 = Takeout(env, 'Customer%d' % 2, counter, 5)
    c3 = Takeout(env, 'Customer%d' % 3, counter, 5)
    
    wt = []
    wt.append(c)
    wt.append(c1)
    wt.append(c2)
    wt.append(c3)
    
    
    i = 4
    while(i):
        c4 = Takeout(env, 'Customer%d' % i, counter, 5)
        wt.append(c4)
        t = random.expovariate(1.0 / interval)
        i+=1
        yield env.timeout(7)
    check = 0
    tt = len(wt)
    print(tt)
    for t in range(len(wt)):
        check+=wt[t].wait
    
    print("Total Waiting Time: ",check)   

In [142]:

counter = simpy.Resource(env, capacity=2)
env.process(Source(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter))
env.run(until=20)



Customer0 arrives at Takeout at 0.00 
Customer0 enters Takeout at  0.00
Customer1 arrives at Takeout at 0.00 
Customer1 enters Takeout at  0.00
Customer2 arrives at Takeout at 0.00 
Customer2 enters Takeout at  0.00
Customer3 arrives at Takeout at 0.00 
Customer3 enters Takeout at  0.00
Customer4 arrives at Takeout at 0.00 
Customer4 enters Takeout at  0.00
Takeout fulfilled 79.16 % of Customer0 s appetite.
Customer0: leaves Takeout at 5.00 
Takeout fulfilled 83.09 % of Customer1 s appetite.
Customer1: leaves Takeout at 5.00 
Takeout fulfilled 61.52 % of Customer2 s appetite.
Customer2: leaves Takeout at 5.00 
Takeout fulfilled 55.87 % of Customer3 s appetite.
Customer3: leaves Takeout at 5.00 
Takeout fulfilled 93.62 % of Customer4 s appetite.
Customer4: leaves Takeout at 5.00 
Customer5 arrives at Takeout at 7.00 
Customer5 enters Takeout at  7.00
Takeout fulfilled 79.13 % of Customer5 s appetite.
Customer5: leaves Takeout at 12.00 
Customer6 arrives at Takeout at 14.00 
Customer6 en

NameError: name 'wt' is not defined